# 1. Hit&Run analysis of our data + import libraries

#### notes:

1. There are some missing values (21%) in the LifeSquare. At the same time they have a "normal" measures in the Square column.
    So, I can change missings on average or median. But, in terms of model improving I'll try to predict them later

2. Actually, i have missing in the Healhcare_1 too (48%). But this column is strange and I don't the logic of it creation.
    Lets look on it distribution later

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error as mse, r2_score as r2

from IPython.display import Image
from sklearn.preprocessing import RobustScaler


from lightgbm import LGBMRegressor

pd.set_option('display.max_columns', None) #to see all columns in the dataset

In [2]:
df = pd.read_csv('train.csv')
X = pd.DataFrame(df)

In [3]:
X.shape

(10000, 20)

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [5]:
X.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [6]:
X.isnull().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

In [7]:
X[X['LifeSquare'].isnull()]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,221244.156664
8,5172,1,2.0,64.511437,NaN,1.0,9,17.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,229102.795999
21,11935,27,2.0,64.711835,NaN,1.0,15,1.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,127200.026511
23,6641,54,3.0,118.907612,NaN,0.0,2,0.0,1977,0.006076,B,B,30,5285,0,645.0,6,6,B,571069.052600
25,5073,74,2.0,82.667915,NaN,1.0,16,17.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,244145.827069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,16159,27,1.0,38.968338,NaN,1.0,10,1.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,95529.569405
9986,2996,1,2.0,64.994245,NaN,1.0,9,22.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,200332.511658
9987,15623,23,2.0,55.620092,NaN,0.0,2,4.0,1977,0.034656,B,B,0,168,0,NaN,0,0,B,115797.553006
9994,4158,23,2.0,64.864198,NaN,1.0,17,17.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,171323.515653


In [8]:
## well, lets create some new binary columns which will indicate a missings in the data
X.loc[X['LifeSquare'].isnull(), 'LifeSquare_missings'] = 1
X.loc[X['Healthcare_1'].isnull(), 'Healthcare_1_missings'] = 1

X["LifeSquare_missings"].fillna(0, inplace=True)
X['Healthcare_1_missings'].fillna(0, inplace=True)

In [9]:
X['Healthcare_1_missings'].dtype

dtype('float64')

In [10]:
## Here I want to remap indexes, cos I already have it in the dataset
X.set_index('Id', inplace=True)

# X.reset_index(inplace=True) - if i'll want to reset an index

# 2. Data Pre-processing

#### notes:

1. Category 'A' in the columns:Ecology_2, Ecology_3, Shops_2 is quite rare. The Price difference in these categories isn't big 

In [11]:
X_objects = X.select_dtypes(include='object')
X_objects

,Ecology_2,Ecology_3,Shops_2
Id,,,
14038,B,B,B
15053,B,B,B
4765,B,B,B
5809,B,B,B
10783,B,B,B
...,...,...,...
77,B,B,B
6159,B,B,A
5123,B,B,A


In [12]:
print(X_objects['Ecology_2'].value_counts())
print(X_objects['Ecology_3'].value_counts())
print(X_objects['Shops_2'].value_counts())

B    9903
A      97
Name: Ecology_2, dtype: int64
B    9725
A     275
Name: Ecology_3, dtype: int64
B    9175
A     825
Name: Shops_2, dtype: int64


In [13]:
groupby = X.groupby('Shops_2')
groupby['Price'].mean()

Shops_2
A    192339.700513
B    216098.999571
Name: Price, dtype: float64

In [14]:
X['Price'].describe()

count     10000.000000
mean     214138.857399
std       92872.293865
min       59174.778028
25%      153872.633942
50%      192269.644879
75%      249135.462171
max      633233.466570
Name: Price, dtype: float64

In [139]:
# ok, first of all, let's replace missings on median or mode (lifeSquare is int, but Healthcare is category, I think)
# anyway, there are a small positive correlation between price and Healthcare_1 (0.13), so I think take a mode would be better,
# cos it is smaller
median_LifeSquare = X['LifeSquare'].median()
mode_Healthcare_1 = X['Healthcare_1'].mode()[0]

X['LifeSquare'] = X['LifeSquare'].fillna(median_LifeSquare)
X['Healthcare_1'] = X['Healthcare_1'].fillna(mode_Healthcare_1)

In [140]:
#X.loc[X['LifeSquare_new'] == X['LifeSquare']]
X['Square']*0.66

Id
14038    31.667830
15053    43.351202
4765     29.665649
5809     35.212968
10783    26.168467
           ...    
77       33.265178
6159     27.404220
5123     31.639745
5400     28.777691
6306     25.519986
Name: Square, Length: 10000, dtype: float64

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 14038 to 6306
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DistrictId             10000 non-null  int64  
 1   Rooms                  10000 non-null  float64
 2   Square                 10000 non-null  float64
 3   LifeSquare             10000 non-null  float64
 4   KitchenSquare          10000 non-null  float64
 5   Floor                  10000 non-null  int64  
 6   HouseFloor             10000 non-null  float64
 7   HouseYear              10000 non-null  int64  
 8   Ecology_1              10000 non-null  float64
 9   Ecology_2              10000 non-null  object 
 10  Ecology_3              10000 non-null  object 
 11  Social_1               10000 non-null  int64  
 12  Social_2               10000 non-null  int64  
 13  Social_3               10000 non-null  int64  
 14  Healthcare_1           10000 non-null  float64
 15 

In [17]:
# next step is work with outliers
X.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,LifeSquare_missings,Healthcare_1_missings
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,50.400800,1.890500,56.315775,36.266040,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,853.630900,1.319500,4.231300,214138.857399,0.211300,0.479800
std,43.587592,0.839512,21.058732,76.609981,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,795.934381,1.493601,4.806341,92872.293865,0.408251,0.499617
min,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028,0.000000,0.000000
25%,20.000000,1.000000,41.774881,25.527399,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,540.000000,0.000000,1.000000,153872.633942,0.000000,0.000000
50%,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,540.000000,1.000000,3.000000,192269.644879,0.000000,0.000000
75%,75.000000,2.000000,65.900625,41.427234,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,990.000000,2.000000,6.000000,249135.462171,0.000000,1.000000
max,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570,1.000000,1.000000


In [18]:
# DistrictId, rooms, square, -don't touch right now

#LifeSquare
X.loc[X['LifeSquare']/X['Square'] > 1, 'LifeSquare'] = X['Square']

#KitchenSquare
X.loc[(X['KitchenSquare'] - X['LifeSquare']) > X['Square'], 'KitchenSquare'] =\
    X.loc[(X['KitchenSquare'] - X['LifeSquare']) > X['Square'], 'Square'] -\
    X.loc[(X['KitchenSquare'] - X['LifeSquare']) > X['Square'], 'LifeSquare']

X.loc[X['KitchenSquare']+X['LifeSquare'] > X['Square'], 'KitchenSquare'] =\
   X.loc[X['KitchenSquare']+X['LifeSquare'] > X['Square'], 'Square'] -\
   X.loc[X['KitchenSquare']+X['LifeSquare'] > X['Square'], 'LifeSquare']

#Floor/HouseFloor
X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor'] = X.loc[X['Floor'] > X['HouseFloor'], 'Floor']

#Year
X.loc[X['HouseYear'] > 2020, 'HouseYear'] = 2020

#Ecology_2, Ecology_3, Shops_2 --> from categorical to numeric:
X.loc[X['Ecology_2'] == 'B', 'Ecology_2'] = 1
X.loc[X['Ecology_2'] == 'A', 'Ecology_2'] = 0

X.loc[X['Ecology_3'] == 'B', 'Ecology_3'] = 1
X.loc[X['Ecology_3'] == 'A', 'Ecology_3'] = 0

X.loc[X['Shops_2'] == 'B', 'Shops_2'] = 1
X.loc[X['Shops_2'] == 'A', 'Shops_2'] = 0

#rooms 
X.loc[X['Rooms'] == 0, 'Rooms'] = 1

# 3. Features Engineering

#### notes:

1. First of all, I'll to start from obvious features
2. I'm not familiar with Ecology, Social, Healthcare and Shops features, so I'll make just a simple transform

In [19]:
# Sqare/Rooms and LifeSquare/Rooms and LifeSquare/Sqare and KitchenSquare/Square
X['Sqare/Rooms'] = X['Square']/X['Rooms']
X['LifeSquare/Rooms'] = X['LifeSquare']/X['Rooms']
X['LifeSquare/Sqare'] = X['LifeSquare']/X['Square']
X['KitchenSquare/Sqare'] = X['KitchenSquare']/X['Square']

X.loc[X['KitchenSquare'] == 0, 'KitchenSquare_is_zero'] = 1
X["KitchenSquare_is_zero"].fillna(0, inplace=True)

#Floor/HouseFloor
X['Floor/HouseFloor'] = X['Floor']/X['HouseFloor']

#HouseYear
X['New_House'] = 2020 - X['HouseYear']

#Ecology, Social, Healthcare and Shops, but firslt - I must to normalize these data

scaler = RobustScaler()
cols_for_scaling = ['Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Shops_2']

X[cols_for_scaling] = scaler.fit_transform(X[cols_for_scaling])
## don't forget to transform a test dataset (scaler.transform)!

X['Sum_Ecology'] = X['Ecology_1'] + X['Ecology_2'] + X['Ecology_3'] 
X['Average_Ecology'] = (X['Ecology_1'] + X['Ecology_2'] + X['Ecology_3']) / 3

X['Sum_Social'] = X['Social_1'] + X['Social_2'] + X['Social_3'] 
X['Average_Social'] = (X['Social_1'] + X['Social_2'] + X['Social_3']) / 3

X['Sum_Healthcare'] = X['Healthcare_1'] + X['Helthcare_2']
X['Average_Healthcare'] = (X['Healthcare_1'] + X['Helthcare_2']) / 2

X['Sum_Shops'] = X['Shops_1'] + X['Shops_2']
X['Average_Shops'] = (X['Shops_1'] + X['Shops_2']) / 2

# 4. Model

#### notes:

The best way is trying to use few models

In [21]:
# firstly, drop price + create a test and train samples
X_final = X.copy()

Y_final = X['Price']
X_final.drop(['Price'], axis=1, inplace=True)

X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, shuffle=True, random_state=42)

In [29]:
# lets try to linear regression
lr_model = LinearRegression()
lr_model.fit(X_train, Y_train)

lr_predict = lr_model.predict(X_test)
r2(Y_test, lr_predict)

0.46827411016830933

In [30]:
# lets try to Decision Tree
lr_model = DecisionTreeRegressor(criterion='mse', 
                                 max_depth=20, # глубина дерева  
                                 min_samples_leaf=30, # минимальное кол-во наблюдений в листе дерева
                                 random_state=42)

lr_model.fit(X_train, Y_train)

lr_predict = lr_model.predict(X_test)
r2(Y_test, lr_predict)

0.6300264424475553

In [33]:
# lets try to Random Forest
lr_model = RandomForestRegressor(criterion='mse',
                                 max_depth=20, # глубина дерева  
                                 min_samples_leaf=30, # минимальное кол-во наблюдений в листе дерева
                                 random_state=42,
                                
                                n_estimators=100  # кол-во деревьев
                                )

lr_model.fit(X_train, Y_train)

lr_predict = lr_model.predict(X_test)
r2(Y_test, lr_predict)

0.6682254536713254

In [39]:
# finally, lets try to Gradient Boosting:
lr_model = LGBMRegressor(criterion='mse',
                        max_depth=7, # Неглубокие < 10
                        min_samples_leaf=10,
                        random_state=42,  
                        
                        n_estimators=100)

lr_model.fit(X_train, Y_train)

lr_predict = lr_model.predict(X_test)
r2(Y_test, lr_predict)

0.7354364722286142

In [61]:
# Tunning:
lgb_model = LGBMRegressor(criterion='mse',
                            # min_samples_leaf=10,
                            random_state=42 #,
                         
                         # n_estimators=100,
                         # max_depth=7
                         
                         )

params = {'min_samples_leaf':[2, 5, 10, 25, 50], 
          'n_estimators':[10, 50, 100, 200, 400], 
          'max_depth':[3, 5, 7, 10, 20]}

gs = GridSearchCV(lgb_model, params, 
                  scoring='r2', # метрика 
                  cv=KFold(n_splits=5,   # k (кол-во разбиений/итераций) в кросс-валидации
                           random_state=21, 
                           shuffle=True), 
                  n_jobs=-1  # Кол-во потоков 
                 )

gs.fit(X_train, Y_train)  # НА TRAIN!

GridSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
             error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0, criterion='mse',
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=42,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 10, 20],
                         'min_samples_leaf': [2, 5

In [62]:
res = pd.DataFrame(gs.cv_results_)
print(gs.best_params_)
print(gs.best_score_)

{'max_depth': 7, 'min_samples_leaf': 2, 'n_estimators': 100}
0.7413488472937406


In [63]:
# final model:
lr_model = LGBMRegressor(criterion='mse',
                        max_depth=7,
                        min_samples_leaf=2,
                        random_state=42,  
                        
                        n_estimators=100)

lr_model.fit(X_train, Y_train)

lr_predict = lr_model.predict(X_test)
r2(Y_test, lr_predict)

0.7354364722286142

In [68]:
# the best features
feature_importances = pd.DataFrame(zip(X_train.columns, 
                                       lr_model.feature_importances_ / lr_model.feature_importances_.sum()), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False, inplace=True)

In [91]:
feature_importances.head(35)

,feature_name,importance
2,Square,0.100946
0,DistrictId,0.088328
7,HouseYear,0.066246
13,Social_3,0.061339
20,Sqare/Rooms,0.056081
8,Ecology_1,0.055380
23,KitchenSquare/Sqare,0.048370
25,Floor/HouseFloor,0.045917
21,LifeSquare/Rooms,0.042762
3,LifeSquare,0.042411


# 5. Validation

In [77]:
df_test = pd.read_csv('test.csv')
X_validation = pd.DataFrame(df_test)

X_validation.loc[X_validation['LifeSquare'].isnull(), 'LifeSquare_missings'] = 1
X_validation.loc[X_validation['Healthcare_1'].isnull(), 'Healthcare_1_missings'] = 1

X_validation["LifeSquare_missings"].fillna(0, inplace=True)
X_validation['Healthcare_1_missings'].fillna(0, inplace=True)

X_validation.set_index('Id', inplace=True)

median_LifeSquare = X_validation['LifeSquare'].median()
mode_Healthcare_1 = X_validation['Healthcare_1'].mode()[0]

X_validation['LifeSquare'] = X_validation['LifeSquare'].fillna(median_LifeSquare)
X_validation['Healthcare_1'] = X_validation['Healthcare_1'].fillna(mode_Healthcare_1)



In [79]:
# DistrictId, rooms, square, -don't touch right now

#LifeSquare
X_validation.loc[X_validation['LifeSquare']/X_validation['Square'] > 1, 'LifeSquare'] = X_validation['Square']

#KitchenSquare
X_validation.loc[(X_validation['KitchenSquare'] - X_validation['LifeSquare']) > X_validation['Square'], 'KitchenSquare'] =\
    X_validation.loc[(X_validation['KitchenSquare'] - X_validation['LifeSquare']) > X_validation['Square'], 'Square'] -\
    X_validation.loc[(X_validation['KitchenSquare'] - X_validation['LifeSquare']) > X_validation['Square'], 'LifeSquare']

X_validation.loc[X_validation['KitchenSquare']+X_validation['LifeSquare'] > X_validation['Square'], 'KitchenSquare'] =\
   X_validation.loc[X_validation['KitchenSquare']+X_validation['LifeSquare'] > X_validation['Square'], 'Square'] -\
   X_validation.loc[X_validation['KitchenSquare']+X_validation['LifeSquare'] > X_validation['Square'], 'LifeSquare']

#Floor/HouseFloor
X_validation.loc[X_validation['Floor'] > X_validation['HouseFloor'], 'HouseFloor'] = X_validation.loc[X_validation['Floor'] > X_validation['HouseFloor'], 'Floor']

#Year
X_validation.loc[X_validation['HouseYear'] > 2020, 'HouseYear'] = 2020

#Ecology_2, Ecology_3, Shops_2 --> from categorical to numeric:
X_validation.loc[X_validation['Ecology_2'] == 'B', 'Ecology_2'] = 1
X_validation.loc[X_validation['Ecology_2'] == 'A', 'Ecology_2'] = 0

X_validation.loc[X_validation['Ecology_3'] == 'B', 'Ecology_3'] = 1
X_validation.loc[X_validation['Ecology_3'] == 'A', 'Ecology_3'] = 0

X_validation.loc[X_validation['Shops_2'] == 'B', 'Shops_2'] = 1
X_validation.loc[X_validation['Shops_2'] == 'A', 'Shops_2'] = 0

#rooms 
X_validation.loc[X_validation['Rooms'] == 0, 'Rooms'] = 1

In [85]:
# Sqare/Rooms and LifeSquare/Rooms and LifeSquare/Sqare and KitchenSquare/Square
X_validation['Sqare/Rooms'] = X_validation['Square']/X_validation['Rooms']
X_validation['LifeSquare/Rooms'] = X_validation['LifeSquare']/X_validation['Rooms']
X_validation['LifeSquare/Sqare'] = X_validation['LifeSquare']/X_validation['Square']
X_validation['KitchenSquare/Sqare'] = X_validation['KitchenSquare']/X_validation['Square']

X_validation.loc[X_validation['KitchenSquare'] == 0, 'KitchenSquare_is_zero'] = 1
X_validation["KitchenSquare_is_zero"].fillna(0, inplace=True)

#Floor/HouseFloor
X_validation['Floor/HouseFloor'] = X_validation['Floor']/X_validation['HouseFloor']

#HouseYear
X_validation['New_House'] = 2020 - X_validation['HouseYear']

#Ecology, Social, Healthcare and Shops, but firslt - I must to normalize these data

scaler = RobustScaler()
cols_for_scaling = ['Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Shops_2']
X[cols_for_scaling] = scaler.fit_transform(X[cols_for_scaling])
X_validation[cols_for_scaling] = scaler.transform(X_validation[cols_for_scaling])

## don't forget to transform a test dataset (scaler.transform)!

X_validation['Sum_Ecology'] = X_validation['Ecology_1'] + X_validation['Ecology_2'] + X_validation['Ecology_3'] 
X_validation['Average_Ecology'] = (X_validation['Ecology_1'] + X_validation['Ecology_2'] + X_validation['Ecology_3']) / 3

X_validation['Sum_Social'] = X_validation['Social_1'] + X_validation['Social_2'] + X_validation['Social_3'] 
X_validation['Average_Social'] = (X_validation['Social_1'] + X_validation['Social_2'] + X_validation['Social_3']) / 3

X_validation['Sum_Healthcare'] = X_validation['Healthcare_1'] + X_validation['Helthcare_2']
X_validation['Average_Healthcare'] = (X_validation['Healthcare_1'] + X_validation['Helthcare_2']) / 2

X_validation['Sum_Shops'] = X_validation['Shops_1'] + X_validation['Shops_2']
X_validation['Average_Shops'] = (X_validation['Shops_1'] + X_validation['Shops_2']) / 2

In [87]:
lr_predict = lr_model.predict(X_validation)

In [90]:
np.savetxt("foo.csv", lr_predict, delimiter=",")